In [121]:
def convRUNKEY(RUN_KEY):
    RUN_KEY = 'v1.0.1_params:Decoder.parameters+encoder.embed.parameters_batch_size:20_vocabThr:4_embedSize:256_hiddenSize:512_totEpochs:3'
    FILENAME = "".join([c for c in RUN_KEY if c not in '( ,:).'])
    return FILENAME

In [122]:
def getLossPreplexityFromLog(filename):
    log = open(filename, 'r').read()
    log = log.split('\n')
    log = [l for l in log if 'Loss:' in l]
    matr_log_ori = [[
        int(l[7]), int(l[19:l.find('/20706')]), 
        float(l[l.find('Loss:') + 6:l.find(', Perplexity:')]), 
        float(l[l.find(', Perplexity:')+14:])] for l in log if len(l) > 19]    
    import numpy as np
    matr_log = np.array(matr_log_ori)
    
    return matr_log
#    x_complete = list(range(len(matr_log)))
#    y_complete = matr_log[:,2]
#    z_complete = matr_log[:,3]

In [193]:
import os
files = os.listdir('numpy')
import pandas as pd
import numpy as np

df = pd.DataFrame()
for f in files:
    a = np.load(open('numpy/' + f, 'rb'))
    df2 = pd.DataFrame(data=a, columns=[f.replace('.np','')])
    df = pd.concat([df, df2])

dfResults = pd.DataFrame(columns=['model', 'batchSize',  'embededSize', 'hiddenSize', 
                                  'measure', 'min', 'mean', 'len','epochs', 'smallDB', 'params', 'key'])
for c in df.columns:
    a = df[c].values
    a = a[~np.isnan(a)]
    c = c.replace(':','')
    subkey = c[c.find('_params') + 7:c.find('_batch_size')]
    batchsize = c[c.find('_batch_size') + 11:c.find('_vocabThr')]
    embedsize = c[c.find('embedSize') + 9:c.find('_hiddenSize')]
    hiddensize = c[c.find('_hiddenSize') + 11:c.find('_totEpochs')]
    measure = 'perplexity' if 'perplexity' in c else 'loss'
    model = c[:c.find('_')]
    lrun = len(a)
    
    smallDB = 'yes' if 'small' in c else 'no'
    epochs = (len(a)*int(batchsize)) // (124230 if smallDB=='yes' else 414113)
    
    newrow = {'model':model, 'batchSize': batchsize,  'embededSize':embedsize,
              'hiddenSize':hiddensize,'measure':measure, 'min': np.min(a),
               'mean':np.mean(a), 'len': len(a), 'epochs':epochs,
              'smallDB':smallDB, 'params':subkey, 'key':c}
    dfResults = dfResults.append(newrow, ignore_index=True)
    print(f"{model:<7}; {batchsize:<3}; {embedsize:<3}; {hiddensize:<4};{measure:<14}; " \
          f"{np.min(a):02.4f}; {np.mean(a):0>7.4f} ({len(a)} - {epochs}); {subkey[:10]}")

v102   ; 10 ; 512; 1024;perplexity    ; 3.3331; 11.9571 (24846 - 2); Decoderpar
v102   ; 20 ; 512; 1024;perplexity    ; 3.7593; 12.1057 (12424 - 2); Decoderpar
v120   ; 10 ; 512; 1024;perplexity    ; 3.9467; 13.6673 (37269 - 3); Decoderpar
v110   ; 20 ; 256; 1024;perplexity    ; 5.3905; 13.7694 (18636 - 3); Decoderpar
v111   ; 20 ; 256; 1024;loss          ; 1.6739; 02.5769 (18636 - 3); Decoderpar
v101   ; 20 ; 256; 512 ;perplexity    ; 5.2345; 10.6808 (12426 - 2); Decoder.pa
v102   ; 10 ; 512; 1024;loss          ; 1.1457; 02.2321 (124236 - 3); Decoderpar
v110   ; 20 ; 256; 1024;loss          ; 1.6846; 02.5527 (18636 - 3); Decoderpar
v102   ; 20 ; 512; 1024;loss          ; 1.3242; 02.2279 (12424 - 2); Decoderpar
v101   ; 20 ; 256; 1024;loss          ; 1.6144; 02.5794 (18636 - 3); Decoderpar
v102   ; 20 ; 512; 1024;loss          ; 1.1927; 02.0776 (18636 - 3); Decoderpar
v102   ; 20 ; 512; 1024;perplexity    ; 3.2961; 10.1748 (18636 - 3); Decoderpar
v102   ; 10 ; 512; 1024;perplexity    ;

In [194]:
dfResults.sort_values(['measure', 'min'])#.loc[dfResults.epochs==3]

,model,batchSize,embededSize,hiddenSize,measure,min,mean,len,epochs,smallDB,params,key
28,v102,10,512,1024,loss,1.037444,2.139891,37269,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
6,v102,10,512,1024,loss,1.145709,2.232116,124236,3,no,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
10,v102,20,512,1024,loss,1.192730,2.077556,18636,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
23,v102,20,512,1024,loss,1.192730,2.077556,18636,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
13,v102,10,512,1024,loss,1.203914,2.269972,24846,2,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
25,v101,20,512,1024,loss,1.235880,2.208368,18636,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v101_paramsDecoderparameters+Encoderparameters...
16,v102,50,512,1024,loss,1.278084,2.102633,7455,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
15,v121,10,512,1024,loss,1.314855,2.426759,37269,3,yes,Decoderparameters+Encoderparameters_withoutFli...,v121_paramsDecoderparameters+Encoderparameters...
8,v102,20,512,1024,loss,1.324242,2.227886,12424,2,yes,Decoderparameters+Encoderparameters_withoutFli...,v102_paramsDecoderparameters+Encoderparameters...
24,v1.0.1,20,256,512,loss,1.329615,2.323899,18636,3,yes,Decoder.parameters+Encoder.parameters_withoutF...,v1.0.1_paramsDecoder.parameters+Encoder.parame...


In [171]:
dfResults.key[0][:len('-perplexity')]

'v102_params'

In [172]:
import os
modfiles = os.listdir('models')

In [178]:
mfiles = [m[:m.find('_totEpochs') + len('_totEpochs')+2] for m in modfiles]

In [184]:
for m in range(len(mfiles)):
    if (mfiles[m][-1] == 'd') or (mfiles[m][-1] == 'e'):
        mfiles[m] = mfiles[m][:-1]

In [190]:
print(*np.unique(mfiles), sep='\n')

.DS_Stor
v100_paramsDecoderparameters+encoderembedparameters_batch_size20_vocabThr4_embedSize256_hiddenSize512_totEpochs3-
v101_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_smallDB_batch_size20_vocabThr4_embedSize256_hiddenSize1024_totEpochs3
v101_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_smallDB_batch_size20_vocabThr4_embedSize256_hiddenSize512_totEpochs3
v101_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_smallDB_batch_size20_vocabThr4_embedSize512_hiddenSize1024_totEpochs3
v101_paramsDecoderparameters+encoderembedparameters_batch_size100_vocabThr4_embedSize256_hiddenSize512_totEpochs3
v102_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_batch_size10_vocabThr4_embedSize512_hiddenSize1024_totEpochs10
v102_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_smallDB_batch_size10_vocabThr4_embedSize512_hiddenSize1024_totEpochs3
v102_paramsDecoderparameters+Encoderparameters_withoutFlipTrans_smallDB_batch_size20_vocabThr4_embedS